In [4]:
import sys
sys.path.insert(0, '../')

In [55]:
import torchtext.data as data
import numpy as np
import training_utils as tu
import models 
from torch.utils.tensorboard import SummaryWriter
from importlib import reload

In [ ]:
import torch

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(3)

In [124]:
TEXT = data.Field(preprocessing=make_padder(4))
LABEL = data.LabelField()
fields = {'category_id': ('label', LABEL), 'lemmatized': ('text', TEXT)}

In [109]:
TEXT.process([['<pad>']])

tensor([[1],
        [1],
        [1],
        [1]])

In [112]:
from torchtext.data import Pipeline

In [116]:
def make_padder(min_len=4):
    def pad(seq):
        if len(seq) >= min_len:
            return seq
        else:
            return seq + ['<unk>'] * (min_len - len(seq))
    return pad

In [125]:
train_data, valid_data = data.TabularDataset.splits(
                                        path = '../data/json',
                                        train = 'train.json',
                                        validation = 'valid.json',
                                        format = 'json',
                                        fields = fields
)


In [81]:

test_data = data.TabularDataset(
                                path = '../data/json/test.json',
                                format = 'json',
                                fields = {'lemmatized': ('text', TEXT)}
)

In [ ]:
import torchtext.vocab as vocab

custom_embeddings = vocab.Vectors(name = '../embeddings/ft_native_300_ru_wiki_lenta_lemmatize.vec',
                                  cache = '../embeddings',
                                  unk_init = torch.Tensor.normal_)

In [38]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors=custom_embeddings)
LABEL.build_vocab(train_data)
print(f'Vocab size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocab size: 25002
Number of classes: 54


In [50]:
BATCH_SIZE = 64

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda ex: len(ex.text),
    sort_within_batch = False,
    device = device)

In [ ]:
from torch.utils import Su

In [145]:
nn.Sequential()(torch.ones(10,10))

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [56]:
reload(models)
from models import CNN
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES,  DROPOUT, PAD_IDX)

In [57]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [58]:
tu.count_parameters(model)

2606754

In [60]:
writer = SummaryWriter('../runs/cnn+embs')

In [ ]:
reload(tu)
tu.train_model(model, train_iterator, valid_iterator,
            optimizer, criterion, 'cnn+embs', n_epochs=25, writer=writer)

Epoch: 1 | Epoch Time: 2m 6:s 


In [139]:
mlp = nn.Sequential()
mlp.add_module('l1', nn.Linear(10,20))
mlp.add_module('a1', nn.ReLU())
mlp.add_module('l1', nn.Linear(20))

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [140]:

sizes = [10,20,30]

In [142]:
[10] + sizes

[10, 10, 20, 30]